#### Импорты

In [ ]:
import momepy
import numpy as np
import osmnx as ox
import networkx as nx
import pandas as pd
import geopandas as gpd
from transport_frames.src.metrics.indicators import get_intermodal
from transport_frames.src.metrics.grade_territory import grade_territory
from transport_frames.src.graph_builder.graphbuilder import get_graph_from_polygon, get_frame, assign_city_names_to_nodes, convert_list_attr_to_str
from transport_frames.src.graph_builder.availability_estimation import availability_matrix, find_median
from transport_frames.src.graph_builder.criteria import weight_territory, assign_grades

#### Загрузка графа

In [ ]:
LOCAL_UTM_CRS = 32636
GLOBAL_CRS = 4326
POLYGON_BUFFER_FOR_ROADS_METERS = 3_000

leningrad_region_poly_osm_relation_id = 'R176095'
saint_petersburg_poly_osm_relation_id = 'R337422'

lo_polygon = ox.geocode_to_gdf(leningrad_region_poly_osm_relation_id, by_osmid=True).to_crs(epsg=LOCAL_UTM_CRS)
spb_polygon = ox.geocode_to_gdf(saint_petersburg_poly_osm_relation_id, by_osmid=True).to_crs(epsg=LOCAL_UTM_CRS).buffer(POLYGON_BUFFER_FOR_ROADS_METERS)
city = lo_polygon.union(spb_polygon).to_crs(epsg=GLOBAL_CRS) #  get lo polygon
russia = ox.geocode_to_gdf("Russia")

citygraph = get_graph_from_polygon(city, crs=LOCAL_UTM_CRS,country_polygon=russia)
citygraph = convert_list_attr_to_str(citygraph)
nx.write_graphml(citygraph, 'spb_roads.graphml')

#### Выделение каркаса

In [ ]:
regions = gpd.read_file('../transport_frames/data/russia.geojson') #  get regions
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions.to_crs(city.crs)

frame = get_frame(citygraph, regions, city)

#### Перенос точек на граф

In [ ]:
cities = gpd.read_file('../transport_frames/data/18_centers.geojson')
frame = assign_city_names_to_nodes(cities.to_crs(frame.graph['crs']), momepy.nx_to_gdf(frame)[0], frame, name_col='name18', max_distance=1200)

#### Оценка территории

In [ ]:
p1 = gpd.read_file('../transport_frames/data/Аэродром лодейнопольское поселение.geojson')
p2 = gpd.read_file('../transport_frames/data/Аэродром Сиверск .geojson')
p3 = gpd.read_file('../transport_frames/data/project Светогорского поселения.geojson')
p4 = gpd.read_file('../transport_frames/data/project Шлиссельбург.geojson')
neudobiya = pd.concat([p1,p2,p3,p4])

In [ ]:
n_grade = grade_territory(neudobiya, frame, include_priority=True)

#### Расчет критериев оценки состояния территории

In [ ]:
polygons188 = gpd.read_file("../transport_frames/data/polygons188.geojson")
points = gpd.read_file("../transport_frames/data/points.geojson")

p1 = gpd.read_file('../transport_frames/data/geojson/Аэродром лодейнопольское поселение.geojson')
p2 = gpd.read_file('../transport_frames/data/geojson/Аэродром Сиверск .geojson')
p3 = gpd.read_file('../transport_frames/data/geojson/project Светогорского поселения.geojson')
p4 = gpd.read_file('../transport_frames/data/geojson/project Шлиссельбург.geojson')

ferry = gpd.read_file('../transport_frames/data/geojson/water_transport_LO.geojson')
aero = gpd.read_file('../transport_frames/data/geojson/airports_local_LO.geojson')
r_stops = gpd.read_file('../transport_frames/data/geojson/ЖД остановки.geojson')
fuel = gpd.read_file('../transport_frames/data/geojson/fuel.geojson')
b_stops = gpd.read_file('../transport_frames/data/geojson/Остановки ЛО.geojson')

In [ ]:
inter = get_intermodal(city_id=, crs=32636)

In [ ]:
adj_mx = availability_matrix(citygraph, points)
p = find_median(points,adj_mx, polygon)

In [ ]:
p_agg = p[p['to_service'] < np.finfo(np.float64).max].copy()
res = gpd.sjoin(p_agg, polygons188, how='left', op='within').groupby('index_right').median(['to_service']).reset_index()
result_df = pd.merge(polygons188.reset_index(), res, left_on='index', right_on='index_right', how='left').drop(columns=['fid_right']).rename(columns={'to_service': 'in_car'})
result_df = result_df.drop(columns=['index_right'])

In [ ]:
adj_mx_inter = availability_matrix(inter,points,graph_type=[GraphType.PUBLIC_TRANSPORT, GraphType.WALK])
p_inter = find_median(points, adj_mx_inter, result_df)
points_inter = p_inter[p_inter['to_service'] < np.finfo(np.float64).max].copy()

In [ ]:
res_inter = gpd.sjoin(points_inter, polygons188, how="left", predicate="within").groupby('index_right').median(['to_service']).reset_index()
result_df_inter = pd.merge(result_df, res_inter, left_on='index', right_on='index_right', how='left').drop(columns=['index_right', 'fid_right']).rename(columns={'to_service': 'in_inter'})

In [ ]:
# Grade the territories
graded_gdf = weight_territory(n_grade, r_stops, b_stops, ferry, aero)

In [ ]:
result = assign_grades(graded_gdf[['name', 'geometry', 'grade', 'weight']], result_df_inter[['index', 'fid', 'name', 'geometry', 'in_car', 'in_inter']])
result.to_file('data/geojson/result_assesment.geojson', driver='GeoJSON')